<a href="https://colab.research.google.com/github/gabrielfernandorey/IEBS/blob/main/01-Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IEBS
### Proyecto fin de Postgrado
--- 
- Preparacion de los datos
- Alumno: Gabriel Rey

## Importamos librerias

In [ ]:
import os
import struct
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import librosa
import librosa.display
from IPython import display as ipd

In [ ]:
# Setear semilla para reproduccion
seed = 42
np.random.seed(seed)

In [ ]:
########### Google Drive #####################
from google.colab import drive

drive.mount('/content/drive/')
DATASET_PATH='/content/drive/MyDrive/Colab Notebooks/CNN/Audio/Pajaros/data'

###### MODO PC LOCAL ############
# DATASET_PATH = 'data'  
# data_dir = pathlib.Path(DATASET_PATH)
# filenames = tf.io.gfile.glob(str(data_dir) + '/*.wav')
# filenames

Mounted at /content/drive/


In [ ]:
filenames = !ls -1 '/content/drive/MyDrive/Colab Notebooks/CNN/Audio/Pajaros/data'
filenames = [f[1:-1] for f in filenames]

In [ ]:
filenames

['0000  RUIDO.wav',
 '0807  BENTEVEO COMÚN.wav',
 '0879  GOLONDRINA DOMÉSTICA.wav',
 '0899  ZORZAL COLORADO.wav',
 '0903  CALANDRIA GRANDE.wav',
 '1062  GORRIÓN.wav',
 '1070  BOYERITO.wav']

In [ ]:
num_samples = len(filenames)
print('Number of total examples:', num_samples)

Number of total examples: 7


### Atributos de los archivos de audio

In [ ]:
class WavFileHelper():
    
    def read_file_properties(self, filename):
        wave_file = open(filename, "rb")

        riff = wave_file.read(12)
        fmt = wave_file.read(36)

        ChunkSizeString = riff[4:8]
        ChunkSize = struct.unpack('I', ChunkSizeString) 
        size = round(ChunkSize[0]/1024)
        
        num_channels_string = fmt[10:12]
        num_channels = struct.unpack('<H', num_channels_string)[0]

        sample_rate_string = fmt[12:16]
        sample_rate = struct.unpack("<I", sample_rate_string)[0]
        
        byte_rate_string = fmt[16:20]
        byte_rate = struct.unpack("<I", byte_rate_string)[0] 

        bit_depth_string = fmt[22:24]
        bit_depth = struct.unpack("<H", bit_depth_string)[0]
        
        timeInSec = size*1024/byte_rate
        
        return (filename, size, num_channels, sample_rate, round(timeInSec,2), bit_depth)

In [ ]:
wavfilehelper = WavFileHelper()

In [ ]:
audiodata = []
for file in filenames:
    data = wavfilehelper.read_file_properties(DATASET_PATH+'/'+file)
    audiodata.append(data)

In [ ]:
# Convert into a Panda dataframe
audiodf = pd.DataFrame(audiodata, columns=['filename','KB','num_channels','sample_rate','timeInSec','bit_depth'])
audiodf

,filename,KB,num_channels,sample_rate,timeInSec,bit_depth
0,/content/drive/MyDrive/Colab Notebooks/CNN/Aud...,12928,1,44100,150.09,16
1,/content/drive/MyDrive/Colab Notebooks/CNN/Aud...,7299,1,44100,84.74,16
2,/content/drive/MyDrive/Colab Notebooks/CNN/Aud...,2151,1,44100,24.97,16
3,/content/drive/MyDrive/Colab Notebooks/CNN/Aud...,13079,1,44100,151.85,16
4,/content/drive/MyDrive/Colab Notebooks/CNN/Aud...,10634,1,44100,123.46,16
5,/content/drive/MyDrive/Colab Notebooks/CNN/Aud...,3710,1,44100,43.07,16
6,/content/drive/MyDrive/Colab Notebooks/CNN/Aud...,2912,1,44100,33.81,16


### Check list - Preprocesamiento

- Normalizar Audios
>- Todos los audios normalizados con igual sample rate
>- Misma potencia promedio (validar)
>- Decidir si los sectores de baja potencia se utilizan o no para entrenar
- Data Generator
>- Duracion de cada bloque
>- MFFC
- Data Augmenation

### Preprocesamiento

### Cargar un audio de ejemplo

In [ ]:
audio, sr = librosa.load(DATASET_PATH+'/'+filenames[0])
start = 22050
stop = start + 44100*2

In [ ]:
start=22050
stop=start+44100*5
plt.figure(figsize=(20,5))
plt.plot(audio)
ipd.Audio(audio, rate=sr)

In [ ]:
def extraer_muestras_gdrive(filename, n_muestras, largo, sr):
    cmap = plt.get_cmap('inferno')
    audio, _ = librosa.load(filename)
    carpeta = re.findall('\d+', filename)[0]
    path_save = '/content/drive/MyDrive/Colab Notebooks/CNN/Audio/Pajaros/audio_imagen/'+carpeta
    os.makedirs(path_save, exist_ok=True)
    
    for n in range(n_muestras):
        start = np.random.randint(1, len(audio)-(largo*sr))
        new_audio = audio[start:start+largo*sr]
        #--------------------------
        plt.figure(figsize=(4, 4))
        spec = np.abs(librosa.stft(new_audio, n_fft = 2048, hop_length = 512))
        dB = librosa.amplitude_to_db(spec, ref = np.max)
        plt.imshow(dB, cmap='hot')
        #--------------------------

        plt.axis('off')
        plt.savefig(path_save+'/'+f'{carpeta}img{n}.png', dpi=128, bbox_inches='tight', pad_inches=0)
        plt.clf()
    
    return

In [ ]:
extraer_muestras_gdrive(DATASET_PATH+'/'+filenames[6], 1000, 10, 22050) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  # This is added back by InteractiveShellApp.init_path()


<Figure size 288x288 with 0 Axes>

<Figure size 288x288 with 0 Axes>

<Figure size 288x288 with 0 Axes>

<Figure size 288x288 with 0 Axes>

...

<Figure size 288x288 with 0 Axes>

<Figure size 288x288 with 0 Axes>

<Figure size 288x288 with 0 Axes>

<Figure size 288x288 with 0 Axes>

<Figure size 288x288 with 0 Axes>